In [45]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [46]:
spark = SparkSession.builder.config("spark.executor.allowSparkContext", "true").appName('learn').getOrCreate()

In [47]:
df = spark.createDataFrame([(i,) for i in range(1000000)], schema=T.StructType([T.StructField('id', T.IntegerType(), True)]))

In [55]:
df2 = df.withColumn('rn', (F.rand()*100).cast(T.IntegerType()))
broadcast_df = F.broadcast(df2)

In [ ]:
spark.conf.set('spark.executor.allowSparkContext', 'true')

In [56]:
from pyspark import SparkContext
def func(cond):
    broadcast_df.createOrReplaceTempView('t')
    
    dff = spark.sql(f'''
        select
            *
        from
            t
        where
            id={cond}
        limit 1
    ''')

    return str(dff.collect()[0][0])

In [57]:
spark.udf.register('fudf', func, T.StringType())

Traceback (most recent call last):
  File "/home/glue_user/spark/python/pyspark/serializers.py", line 458, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/home/glue_user/spark/python/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/home/glue_user/spark/python/pyspark/cloudpickle/cloudpickle_fast.py", line 602, in dump
    return Pickler.dump(self, obj)
  File "/home/glue_user/spark/python/pyspark/context.py", line 455, in __getnewargs__
    raise RuntimeError(
RuntimeError: It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.


PicklingError: Could not serialize object: RuntimeError: It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.

In [54]:
df.createOrReplaceTempView('df')
spark.sql('''
select
    fudf(id)
from
    df
''').show()

23/04/09 19:45:43 WARN TaskSetManager: Stage 0 contains a task of very large size (1590 KiB). The maximum recommended task size is 1000 KiB.
23/04/09 19:45:46 ERROR Executor: Exception in task 0.0 in stage 0.0 (TID 0)/ 1]
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/ipykernel_17363/2941741573.py", line 3, in func
  File "/home/glue_user/spark/python/lib/pyspark.zip/pyspark/context.py", line 491, in getOrCreate
    SparkContext(conf=conf or SparkConf())
  File "/home/glue_user/spark/python/lib/pyspark.zip/pyspark/context.py", line 186, in __init__
    SparkContext._assert_on_driver()
  File "/home/glue_user/spark/python/lib/pyspark.zip/pyspark/context.py", line 1541, in _assert_on_driver
    raise RuntimeError("SparkContext should only be created and accessed on the driver.")
RuntimeError: SparkContext should only be created and accessed on the driver.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(P

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/tmp/ipykernel_17363/2941741573.py", line 3, in func
  File "/home/glue_user/spark/python/lib/pyspark.zip/pyspark/context.py", line 491, in getOrCreate
    SparkContext(conf=conf or SparkConf())
  File "/home/glue_user/spark/python/lib/pyspark.zip/pyspark/context.py", line 186, in __init__
    SparkContext._assert_on_driver()
  File "/home/glue_user/spark/python/lib/pyspark.zip/pyspark/context.py", line 1541, in _assert_on_driver
    raise RuntimeError("SparkContext should only be created and accessed on the driver.")
RuntimeError: SparkContext should only be created and accessed on the driver.


In [ ]:
import random
def rn():
    random_int = random.randint(1, 100)
    return int(random_int)

rn_udf = F.udf(rn, T.IntegerType())
spark.udf.register('rn_udf', rn_udf)

In [ ]:
spark.range(100).withColumn('rn', rn_udf()).show()

In [ ]:
spark.range(100).withColumn('a', rn_udf()).show()

In [ ]:
rn_udf()

In [ ]:
df = spark.range(1000000).withColumn('rn', rn())

In [ ]:
df.show()

In [ ]:
rnd()

In [ ]:
df = spark.createDataFrame(data=[], schema=T.StructType([T.StructField("name", T.StringType()), T.StructField("age", T.IntegerType()), T.StructField("gender", T.StringType())]))


In [ ]:
df.count()

In [ ]:
df_p = df.groupBy('partition_id').count()


In [ ]:
df_p.printSchema()

In [ ]:
df.columns

In [ ]:
df.withColumn('fullname', F.concat(
    F.col('first_name'), F.col('last_name'))).show()


In [44]:
spark.stop()